# Dennis Pierantozzi & Francesco Maria Mosca

# Practice 1 CNNs

## Part 1

### Loading the dataset

In [4]:
import os

# Get the current working directory
current_dir = os.getcwd()

# Define the relative paths to your train and test directories
train_dir = os.path.join(current_dir, 'animals', 'train')
test_dir = os.path.join(current_dir, 'animals', 'val')

### Preprocessing the data

What have we done here? <br>
We have used the function the professor suggested and preprocessed the data <br>
for the train data the labels are inferred by the directory and categorical encoded. Size fixed in 120 and 120 and shuffled <br>
for the test same but not shuffled since it is not needed <br>
#### Created a validation dataset from the training dataset (split: 0.2)

In [5]:
from keras.utils import image_dataset_from_directory

# Create TensorFlow datasets for training and validation
train_data = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',  # For one-hot encoded labels
    image_size=(120, 120),  # Resize images to (120, 120)
    batch_size=32,  # Batch size
    shuffle=True,  # Shuffle the data
    seed=123,  # Random seed for shuffling
    validation_split=0.2,  # Fraction of training data to use for validation
    subset='training'  # Use the training subset
)

validation_data = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',  # For one-hot encoded labels
    image_size=(120, 120),  # Resize images to (120, 120)
    batch_size=32,  # Batch size
    shuffle=False,  # Do not shuffle the data
    seed=123,  # Random seed for shuffling
    validation_split=0.2,  # Fraction of training data to use for validation
    subset='validation'  # Use the validation subset
)

test_data = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',  # For one-hot encoded labels
    image_size=(120, 120),  # Resize images to (120, 120)
    batch_size=32  # Batch size
)

Found 13474 files belonging to 5 classes.
Using 10780 files for training.
Found 13474 files belonging to 5 classes.
Using 2694 files for validation.
Found 1497 files belonging to 5 classes.


### Data agumentation

Why data agumentation? Which kind of things are we doing here? 

In [6]:
import keras
from keras import layers
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
        layers.Rescaling(1./255)
    ]
)

## Part 2

### Custom convolutional model

Our first idea has been to add 3 convulational layers with increasing filters. Kernel size 3 as in the lecture examples without padding or strides. <br> Then two pooling layers to reduce feature complexity

In [8]:
from tensorflow import keras 
from tensorflow.keras import layers

inputs = keras.Input(shape=(120, 120, 3))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
outputs = layers.Dense(5, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [11]:
model.compile(optimizer="rmsprop",
    loss="categorical_crossentropy",
    metrics=["accuracy"])

In [12]:
# Fit the model
history = model.fit(
    train_data,
    validation_data=validation_data,
    epochs=10  # Adjust the number of epochs as needed
)

Epoch 1/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 171s 500ms/step - accuracy: 0.3781 - loss: 16.1459 - val_accuracy: 0.4488 - val_loss: 1.3307
Epoch 2/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 139s 411ms/step - accuracy: 0.7053 - loss: 0.8447 - val_accuracy: 0.7903 - val_loss: 0.7328
Epoch 3/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 128s 379ms/step - accuracy: 0.7593 - loss: 0.6590 - val_accuracy: 0.6756 - val_loss: 0.9648
Epoch 4/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 135s 399ms/step - accuracy: 0.8121 - loss: 0.5151 - val_accuracy: 0.9094 - val_loss: 0.3597
Epoch 5/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 124s 367ms/step - accuracy: 0.8435 - loss: 0.4440 - val_accuracy: 0.6255 - val_loss: 1.1357
Epoch 6/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 122s 360ms/step - accuracy: 0.8504 - loss: 0.4360 - val_accuracy: 0.9395 - val_loss: 0.2570
Epoch 7/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 124s 366ms/step - accuracy: 0.8821 - loss: 0.3563 - val_accuracy: 0.7910 - val_loss: 0.7823
Epoch 8/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 123s 364ms/step - accuracy: 0.8873 